# Examples

The following examples show how to use the loader APIs:

## 1. Parsing

This sample shows how to read a CWL document from a remote public URL:

In [1]:
from cwl_loader import load_cwl_from_location

cwl_document = load_cwl_from_location('https://raw.githubusercontent.com/eoap/schemas/main/experimental/stac-api-client.cwl')

2025-08-25 14:53:15.728 | DEBUG    | cwl_loader:load_cwl_from_location:196 - Loading CWL document from https://raw.githubusercontent.com/eoap/schemas/main/experimental/stac-api-client.cwl...
2025-08-25 14:53:16.145 | DEBUG    | cwl_loader:_load_cwl_from_stream:199 - Reading stream from https://raw.githubusercontent.com/eoap/schemas/main/experimental/stac-api-client.cwl...
2025-08-25 14:53:16.150 | DEBUG    | cwl_loader:load_cwl_from_stream:173 - CWL data of type <class 'ruamel.yaml.comments.CommentedMap'> successfully loaded from stream
2025-08-25 14:53:16.150 | DEBUG    | cwl_loader:load_cwl_from_yaml:124 - Updating the model of type 'CommentedMap' to version 'v1.2'...
2025-08-25 14:53:16.153 | DEBUG    | cwl_loader:load_cwl_from_yaml:135 - Raw CWL document successfully updated to v1.2! Now converting to the CWL model...
2025-08-25 14:53:36.418 | DEBUG    | cwl_loader:load_cwl_from_yaml:143 - Raw CWL document successfully updated to v1.2! Now dereferencing the FQNs...
2025-08-25 14:53

## 2. Serializing

This sample shows how to write a CWL document to a stream (string, file, ...):

In [2]:
from cwl_loader import dump_cwl
import sys

dump_cwl(process=cwl_document, stream=sys.stderr)

id: stac-client
class: CommandLineTool
label: STAC Client Tool
doc: |
  This tool uses the STAC Client to search for STAC items
inputs:
- id: api_endpoint
  label: STAC API endpoint
  doc: STAC API endpoint for Landsat-9 data
  type: |-
    https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml#APIEndpoint
- id: search_request
  label: STAC API settings
  doc: STAC API settings for Landsat-9 data
  type: |-
    https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml#STACSearchSettings
outputs:
- id: search_output
  type: File
  outputBinding:
    glob: discovery-output.json
requirements:
- class: InlineJavascriptRequirement
- class: NetworkAccess
  networkAccess: true
- class: SchemaDefRequirement
  types:
  - name: 
      https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml#Date
    fields:
    - name: 
        https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml#Date/value
      type: string
    t

## 3. Parsing existing data structures

Users can load CWL Worflow(s) even from existing dictionaries:

In [3]:
from cwl_loader import load_cwl_from_yaml

cwl_document = load_cwl_from_yaml(raw_process={
  "cwlVersion": "v1.2", 
  "inputs": {
    "api_endpoint": {
      "doc": "STAC API endpoint for Landsat-9 data", 
      "type": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml#APIEndpoint", 
      "label": "STAC API endpoint"
    }, 
    "search_request": {
      "doc": "STAC API settings for Landsat-9 data", 
      "type": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml#STACSearchSettings", 
      "label": "STAC API settings"
    }
  }, 
  "requirements": [
    {
      "class": "InlineJavascriptRequirement"
    }, 
    {
      "class": "NetworkAccess", 
      "networkAccess": True
    }, 
    {
      "class": "SchemaDefRequirement", 
      "types": [
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml"
        }, 
        {
          "$import": "https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml"
        }
      ]
    }
  ], 
  "doc": "This tool uses the STAC Client to search for STAC items\n", 
  "class": "CommandLineTool", 
  "baseCommand": [
    "stac-client"
  ], 
  "label": "STAC Client Tool", 
  "arguments": [
    "search", 
    "$(inputs.api_endpoint.url.value)", 
    "${ const args = []; const collections = inputs.search_request.collections; args.push('--collections', collections.join(\",\")); return args; }", 
    "${ const args = []; const bbox = inputs.search_request?.bbox; if (Array.isArray(bbox) && bbox.length >= 4) { args.push('--bbox', ...bbox.map(String)); } return args; }", 
    "${ const args = []; const limit = inputs.search_request?.limit; args.push(\"--limit\", (limit ?? 10).toString()); return args; }", 
    "${ const maxItems = 5; return ['--max-items', maxItems.toString()]; }", 
    "${ const args = []; const filter = inputs.search_request?.filter; const filterLang = inputs.search_request?.['filter-lang']; if (filterLang) { args.push('--filter-lang', filterLang); } if (filter) { args.push('--filter', JSON.stringify(filter)); } return args; }", 
    "${ const datetime = inputs.search_request?.datetime; const datetimeInterval = inputs.search_request?.datetime_interval; if (datetime) { return ['--datetime', datetime]; } else if (datetimeInterval) { const start = datetimeInterval.start?.value || '..'; const end = datetimeInterval.end?.value || '..'; return ['--datetime', `${start}/${end}`]; } return []; }", 
    "${ const ids = inputs.search_request?.ids; const args = []; if (Array.isArray(ids) && ids.length > 0) { args.push('--ids', ...ids.map(String)); } return args; }", 
    "${ const intersects = inputs.search_request?.intersects; if (intersects) { return ['--intersects', JSON.stringify(intersects)]; } return []; }", 
    "--save", 
    "discovery-output.json"
  ], 
  "outputs": {
    "search_output": {
      "outputBinding": {
        "glob": "discovery-output.json"
      }, 
      "type": "File"
    }
  }, 
  "id": "stac-client", 
  "hints": [
    {
      "dockerPull": "docker.io/library/stac-client", 
      "class": "DockerRequirement"
    }
  ]
})

dump_cwl(process=cwl_document, stream=sys.stderr)

2025-08-25 14:53:36.463 | DEBUG    | cwl_loader:load_cwl_from_yaml:124 - Updating the model of type 'dict' to version 'v1.2'...
2025-08-25 14:53:36.464 | DEBUG    | cwl_loader:load_cwl_from_yaml:135 - Raw CWL document successfully updated to v1.2! Now converting to the CWL model...
2025-08-25 14:53:50.200 | DEBUG    | cwl_loader:load_cwl_from_yaml:143 - Raw CWL document successfully updated to v1.2! Now dereferencing the FQNs...
2025-08-25 14:53:50.200 | DEBUG    | cwl_loader:_clean_process:66 -   Cleaning CommandLineTool stac-client...
2025-08-25 14:53:50.201 | DEBUG    | cwl_loader:load_cwl_from_yaml:151 - CWL document successfully dereferenced!
id: stac-client
class: CommandLineTool
label: STAC Client Tool
doc: "This tool uses the STAC Client to search for STAC items\n"
inputs:
- id: api_endpoint
  label: STAC API endpoint
  doc: STAC API endpoint for Landsat-9 data
  type: 
    https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml#APIEndpoint
- id: sear